In [175]:
import pandas as pd

sitibike = pd.read_csv('citibike-tripdata.csv', sep=',')

In [150]:
#Сколько пропусков в столбце start station id?
sitibike['start station id'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 300000 entries, 0 to 299999
Series name: start station id
Non-Null Count   Dtype  
--------------   -----  
299831 non-null  float64
dtypes: float64(1)
memory usage: 2.3 MB


In [151]:
sitibike['starttime'].info()
sitibike['stoptime'].info()


<class 'pandas.core.series.Series'>
RangeIndex: 300000 entries, 0 to 299999
Series name: starttime
Non-Null Count   Dtype 
--------------   ----- 
300000 non-null  object
dtypes: object(1)
memory usage: 2.3+ MB
<class 'pandas.core.series.Series'>
RangeIndex: 300000 entries, 0 to 299999
Series name: stoptime
Non-Null Count   Dtype 
--------------   ----- 
300000 non-null  object
dtypes: object(1)
memory usage: 2.3+ MB


In [152]:
sitibike['bikeid'].mode()

0    33887
Name: bikeid, dtype: int64

In [153]:
sitibike['usertype'].value_counts(normalize=True)

Subscriber    0.774007
Customer      0.225993
Name: usertype, dtype: float64

In [154]:
sitibike['gender'].value_counts()

1    183582
2     74506
0     41912
Name: gender, dtype: int64

In [155]:
sitibike.describe()

,start station id,start station latitude,start station longitude,end station id,end station latitude,end station longitude,bikeid,birth year,gender
count,299831.000000,300000.000000,300000.000000,299831.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000
mean,1695.665665,40.737769,-73.982042,1684.624298,40.737294,-73.982330,26688.525943,1979.364803,1.108647
std,1458.264055,0.031999,0.020207,1457.549589,0.031794,0.020261,6271.691143,11.920796,0.613398
min,72.000000,40.655400,-74.025353,72.000000,40.655400,-74.083639,14529.000000,1885.000000,0.000000
25%,387.000000,40.716226,-73.995991,387.000000,40.716059,-73.996621,20569.000000,1969.000000,1.000000
50%,519.000000,40.737262,-73.985162,519.000000,40.737050,-73.985649,28197.000000,1982.000000,1.000000
75%,3282.000000,40.760683,-73.970416,3282.000000,40.760094,-73.971092,32216.000000,1989.000000,1.000000
max,3705.000000,40.869000,-73.878000,3705.000000,40.869000,-73.878000,34863.000000,2002.000000,2.000000


In [156]:
display(sitibike['start station name'].value_counts())


Grand Army Plaza & Central Park S    1928
Central Park S & 6 Ave               1909
Pershing Square North                1873
12 Ave & W 40 St                     1845
West St & Chambers St                1835
                                     ... 
NYCBS Depot - GOW                      15
Franklin Ave & Empire Blvd             11
Railroad Ave & Kay Ave                  9
47 Ave & Skillman Ave                   7
Eastern Pkwy & Washington Ave           6
Name: start station name, Length: 759, dtype: int64

In [157]:
display(sitibike['end station name'].value_counts())

West St & Chambers St          1963
12 Ave & W 40 St               1909
Pershing Square North          1850
Central Park S & 6 Ave         1828
E 17 St & Broadway             1804
                               ... 
Exchange Place                    3
Union St                          1
Warren St                         1
Montrose Ave & Bushwick Ave       1
Liberty Light Rail                1
Name: end station name, Length: 765, dtype: int64

In [158]:

sitibike = sitibike.drop(['start station id', 'end station id'], axis=1)
sitibike.shape

(300000, 12)

In [159]:
##Замените признак birth year на более понятный признак возраста клиента age. Годом отсчёта возраста выберите 2018 год. 
sitibike['age'] = 2018 - sitibike['birth year']
display(sitibike['age'])

0         38
1         49
2         43
3         34
4         24
          ..
299995    28
299996    36
299997    27
299998    43
299999    52
Name: age, Length: 300000, dtype: int64

In [176]:
#В наших данных присутствуют столбцы, которые дублируют информацию друг о друге:
# это столбцы с идентификатором и названием стартовой и конечной стоянки.
# Удалите признаки идентификаторов стоянок. Сколько столбцов осталось?
sitibike = sitibike.drop(['birth year'], axis = 1)


In [161]:
#Сколько поездок совершено клиентами старше 60 лет?
agge = sitibike[sitibike['age'] > 60]
agge.shape[0]

11837

In [162]:
#Создайте признак длительности поездки trip duration. Для этого вычислите интервал времени между временем окончания поездки 
# (stoptime) и временем её начала (starttime) в секундах.
# Рассчитайте среднее значение по новому столбцу — среднюю длительность поездки, а затем переведите её в секунды
sitibike['stoptime'] =pd.to_datetime(sitibike['stoptime'], dayfirst=True)
sitibike['starttime'] =pd.to_datetime(sitibike['starttime'], dayfirst=True)

sitibike['trip duration'] = sitibike['stoptime'] - sitibike['starttime']




print(sitibike['trip duration'].mean().seconds)

1126


In [163]:
#Создайте «признак-мигалку» weekend, который равен 1, если поездка начиналась в выходной день (суббота или воскресенье),
# и 0 — в противном случае. Выясните, сколько поездок начиналось в выходные.
weekday = sitibike['starttime'].dt.dayofweek
sitibike['weekend'] = weekday.apply(lambda x: 1 if x ==5 or x == 6 else 0)
sitibike['weekend'].sum()


115135

In [173]:
#Создайте признак времени суток поездки time_of_day. Время суток будем определять из часа начала поездки
#Во сколько раз количество поездок, совершённых днём, больше, чем  ночью?
def get_time_of_day(time):
    if 0 <= time <= 6:
        return 'night'
    elif 6 < time <= 12:
        return 'morning'
    elif 12 < time <= 18:
        return 'day'
    elif 18 < time <= 23:
        return 'evening'
    else:
        return 'else'
sitibike['time_of_day'] = sitibike['starttime'].dt.hour.apply(get_time_of_day)
a = sitibike[sitibike['time_of_day'] == 'day'].shape[0]
b = sitibike[sitibike['time_of_day'] == 'night'].shape[0]
print(round(a / b))

9
